# Unity ML Agents
## Proximal Policy Optimization (PPO)
Contains an implementation of PPO as described [here](https://arxiv.org/abs/1707.06347).

In [1]:
import numpy as np
import os
import tensorflow as tf

from ppo.history import *
from ppo.models import *
from ppo.trainer import Trainer
from unityagents import *

### Hyperparameters

In [3]:
### General parameters
max_steps = 8e6 # Set maximum number of steps to run environment.
run_path = "ppo" # The sub-directory name for model and summary statistics
load_model = False # Whether to load a saved model.
train_model = True # Whether to train the model.
summary_freq = 10000 # Frequency at which to save training statistics.
save_freq = 50000 # Frequency at which to save model.
env_name = "Runner" # Name of the training environment file.
curriculum_file = None

### Algorithm-specific parameters for tuning
gamma = 0.99 # Reward discount rate.
lambd = 0.95 # Lambda parameter for GAE.
time_horizon = 2048 # How many steps to collect per agent before adding to buffer.
beta = 1e-3 # Strength of entropy regularization
num_epoch = 5 # Number of gradient descent steps per batch of experiences.
num_layers = 2 # Number of hidden layers between state/observation encoding and value/policy layers.
epsilon = 0.2 # Acceptable threshold around ratio of old and new policy probabilities.
buffer_size = 2048 # How large the experience buffer should be before gradient descent.
learning_rate = 3e-4 # Model learning rate.
hidden_units = 16 # Number of units in hidden layer.
batch_size = 64 # How many experiences per gradient descent update step.
normalize = False

### Logging dictionary for hyperparameters
hyperparameter_dict = {'max_steps':max_steps, 'run_path':run_path, 'env_name':env_name,
    'curriculum_file':curriculum_file, 'gamma':gamma, 'lambd':lambd, 'time_horizon':time_horizon,
    'beta':beta, 'num_epoch':num_epoch, 'epsilon':epsilon, 'buffe_size':buffer_size,
    'leaning_rate':learning_rate, 'hidden_units':hidden_units, 'batch_size':batch_size}

### Load the environment

In [4]:
env = UnityEnvironment(file_name=env_name, curriculum=curriculum_file)
print(str(env))
brain_name = env.external_brain_names[0]

INFO:unityagents:
'RunnerAcademy' started successfully!


Unity Academy name: RunnerAcademy
        Number of brains: 1
        Reset Parameters :
		
Unity brain name: RunnerBrain
        Number of observations (per agent): 0
        State space type: continuous
        State space size (per agent): 5
        Action space type: discrete
        Action space size (per agent): 2
        Memory space size (per agent): 0
        Action descriptions: NotJump, Jump


### Train the Agent(s)

In [5]:
tf.reset_default_graph()

if curriculum_file == "None":
    curriculum_file = None


def get_progress():
    if curriculum_file is not None:
        if env._curriculum.measure_type == "progress":
            return steps / max_steps
        elif env._curriculum.measure_type == "reward":
            return last_reward
        else:
            return None
    else:
        return None

# Create the Tensorflow model graph
ppo_model = create_agent_model(env, lr=learning_rate,
                               h_size=hidden_units, epsilon=epsilon,
                               beta=beta, max_step=max_steps, 
                               normalize=normalize, num_layers=num_layers)

is_continuous = (env.brains[brain_name].action_space_type == "continuous")
use_observations = (env.brains[brain_name].number_observations > 0)
use_states = (env.brains[brain_name].state_space_size > 0)

model_path = './models/{}'.format(run_path)
summary_path = './summaries/{}'.format(run_path)

if not os.path.exists(model_path):
    os.makedirs(model_path)

if not os.path.exists(summary_path):
    os.makedirs(summary_path)

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    # Instantiate model parameters
    if load_model:
        print('Loading Model...')
        ckpt = tf.train.get_checkpoint_state(model_path)
        saver.restore(sess, ckpt.model_checkpoint_path)
    else:
        sess.run(init)
    steps, last_reward = sess.run([ppo_model.global_step, ppo_model.last_reward])    
    summary_writer = tf.summary.FileWriter(summary_path)
    info = env.reset(train_mode=train_model, progress=get_progress())[brain_name]
    trainer = Trainer(ppo_model, sess, info, is_continuous, use_observations, use_states, train_model)
    if train_model:
        trainer.write_text(summary_writer, 'Hyperparameters', hyperparameter_dict, steps)
    while steps <= max_steps:
        if env.global_done:
            info = env.reset(train_mode=train_model, progress=get_progress())[brain_name]
        # Decide and take an action
        new_info = trainer.take_action(info, env, brain_name, steps, normalize)
        info = new_info
        trainer.process_experiences(info, time_horizon, gamma, lambd)
        if len(trainer.training_buffer['actions']) > buffer_size and train_model:
            # Perform gradient descent with experience buffer
            trainer.update_model(batch_size, num_epoch)
        if steps % summary_freq == 0 and steps != 0 and train_model:
            # Write training statistics to tensorboard.
            trainer.write_summary(summary_writer, steps, env._curriculum.lesson_number)
        if steps % save_freq == 0 and steps != 0 and train_model:
            # Save Tensorflow model
            save_model(sess, model_path=model_path, steps=steps, saver=saver)
        steps += 1
        sess.run(ppo_model.increment_step)
        if len(trainer.stats['cumulative_reward']) > 0:
            mean_reward = np.mean(trainer.stats['cumulative_reward'])
            sess.run(ppo_model.update_reward, feed_dict={ppo_model.new_reward: mean_reward})
            last_reward = sess.run(ppo_model.last_reward)
    # Final save Tensorflow model
    if steps != 0 and train_model:
        save_model(sess, model_path=model_path, steps=steps, saver=saver)
env.close()
export_graph(model_path, env_name)

Step: 10000. Mean Reward: -0.018047882136279954. Std of Reward: 0.9720761805733418.
Step: 20000. Mean Reward: 0.2759615384615384. Std of Reward: 1.1796400505537104.
Step: 30000. Mean Reward: 0.6925249169435216. Std of Reward: 1.5113064259661313.
Step: 40000. Mean Reward: 1.1227272727272728. Std of Reward: 1.7716859175188984.
Step: 50000. Mean Reward: 1.2877216916780354. Std of Reward: 1.7869370225456962.
Saved Model
Step: 60000. Mean Reward: 1.8718792866941016. Std of Reward: 1.9081564603918149.
Step: 70000. Mean Reward: 2.0207496653279784. Std of Reward: 1.9274468550649122.
Step: 80000. Mean Reward: 1.9873357228195938. Std of Reward: 1.7529411225270692.
Step: 90000. Mean Reward: 2.1304054054054054. Std of Reward: 1.8337438932722552.
Step: 100000. Mean Reward: 2.164810281517748. Std of Reward: 1.7344326991129715.
Saved Model
Step: 110000. Mean Reward: 2.282439335887612. Std of Reward: 1.7996721855844617.
Step: 120000. Mean Reward: 2.252948717948718. Std of Reward: 1.8573151348628654.
S

Step: 1000000. Mean Reward: 13.699074074074074. Std of Reward: 7.877776743892389.
Saved Model
Step: 1010000. Mean Reward: 13.147256097560973. Std of Reward: 8.37814602914234.
Step: 1020000. Mean Reward: 12.272701149425286. Std of Reward: 7.950718477784708.
Step: 1030000. Mean Reward: 13.302760736196321. Std of Reward: 8.215125679157193.
Step: 1040000. Mean Reward: 12.474561403508773. Std of Reward: 8.15492777588222.
Step: 1050000. Mean Reward: 13.20670731707317. Std of Reward: 7.911080767779881.
Saved Model
Step: 1060000. Mean Reward: 12.5. Std of Reward: 7.665901229905501.
Step: 1070000. Mean Reward: 12.993072289156625. Std of Reward: 7.897763345545633.
Step: 1080000. Mean Reward: 13.54940119760479. Std of Reward: 8.123187016565995.
Step: 1090000. Mean Reward: 13.500937500000001. Std of Reward: 8.158435955873758.
Step: 1100000. Mean Reward: 12.389457831325302. Std of Reward: 7.611980906473094.
Saved Model
Step: 1110000. Mean Reward: 12.084782608695651. Std of Reward: 7.904537268227072

Step: 1990000. Mean Reward: 15.202173913043477. Std of Reward: 7.916072469895424.
Step: 2000000. Mean Reward: 16.13344370860927. Std of Reward: 8.007163022197675.
Saved Model
Step: 2010000. Mean Reward: 13.610606060606061. Std of Reward: 8.230372037466763.
Step: 2020000. Mean Reward: 15.514423076923077. Std of Reward: 8.316376355889672.
Step: 2030000. Mean Reward: 14.662578616352201. Std of Reward: 8.294957490296865.
Step: 2040000. Mean Reward: 14.016666666666666. Std of Reward: 8.29434211442421.
Step: 2050000. Mean Reward: 13.867592592592594. Std of Reward: 8.36277479244066.
Saved Model
Step: 2060000. Mean Reward: 14.164779874213835. Std of Reward: 8.00847929851948.
Step: 2070000. Mean Reward: 15.664814814814815. Std of Reward: 7.978528069192438.
Step: 2080000. Mean Reward: 14.374844720496897. Std of Reward: 8.32007996151181.
Step: 2090000. Mean Reward: 14.485000000000003. Std of Reward: 8.224053516970814.
Step: 2100000. Mean Reward: 15.325471698113208. Std of Reward: 8.04849856681459

Step: 2970000. Mean Reward: 14.897756410256411. Std of Reward: 8.08877891435286.
Step: 2980000. Mean Reward: 17.40286624203822. Std of Reward: 7.373619361296968.
Step: 2990000. Mean Reward: 16.004605263157895. Std of Reward: 8.054763945181998.
Step: 3000000. Mean Reward: 16.54070512820513. Std of Reward: 7.892947201675382.
Saved Model
Step: 3010000. Mean Reward: 16.56045751633987. Std of Reward: 8.02699622636823.
Step: 3020000. Mean Reward: 16.405228758169933. Std of Reward: 8.071946018734828.
Step: 3030000. Mean Reward: 16.494444444444444. Std of Reward: 7.663397759103584.
Step: 3040000. Mean Reward: 16.972115384615382. Std of Reward: 7.282073705217822.
Step: 3050000. Mean Reward: 15.833333333333334. Std of Reward: 7.5909724836718535.
Saved Model
Step: 3060000. Mean Reward: 16.736274509803923. Std of Reward: 6.998882346102055.
Step: 3070000. Mean Reward: 16.16955128205128. Std of Reward: 7.435544773633166.
Step: 3080000. Mean Reward: 17.35. Std of Reward: 6.859229255100456.
Step: 3090

Step: 3950000. Mean Reward: 17.56078431372549. Std of Reward: 7.115404767528478.
Saved Model
Step: 3960000. Mean Reward: 18.640064102564104. Std of Reward: 6.8414845533207735.
Step: 3970000. Mean Reward: 17.970129870129867. Std of Reward: 6.471938727477621.
Step: 3980000. Mean Reward: 17.986513157894734. Std of Reward: 7.165386394608276.
Step: 3990000. Mean Reward: 19.476797385620912. Std of Reward: 6.465246524230775.
Step: 4000000. Mean Reward: 17.204248366013072. Std of Reward: 6.992563621710839.
Saved Model
Step: 4010000. Mean Reward: 18.922903225806454. Std of Reward: 6.363796895416966.
Step: 4020000. Mean Reward: 17.916666666666668. Std of Reward: 7.151664644650396.
Step: 4030000. Mean Reward: 19.03774834437086. Std of Reward: 6.280849604252515.
Step: 4040000. Mean Reward: 18.096794871794874. Std of Reward: 6.559507387224334.
Step: 4050000. Mean Reward: 19.00718954248366. Std of Reward: 6.068512481268821.
Saved Model
Step: 4060000. Mean Reward: 17.59383116883117. Std of Reward: 7.

Step: 4930000. Mean Reward: 17.91348684210526. Std of Reward: 7.076141274870939.
Step: 4940000. Mean Reward: 17.104545454545455. Std of Reward: 7.035636015602128.
Step: 4950000. Mean Reward: 17.684313725490195. Std of Reward: 7.262949828774931.
Saved Model
Step: 4960000. Mean Reward: 17.42581699346405. Std of Reward: 7.287089273787968.
Step: 4970000. Mean Reward: 17.947058823529414. Std of Reward: 7.434184500845954.
Step: 4980000. Mean Reward: 18.21797385620915. Std of Reward: 7.216085006625539.
Step: 4990000. Mean Reward: 17.433774834437084. Std of Reward: 6.915304913162954.
Step: 5000000. Mean Reward: 18.001612903225805. Std of Reward: 7.294997002707467.
Saved Model
Step: 5010000. Mean Reward: 17.829807692307693. Std of Reward: 6.848705611483189.
Step: 5020000. Mean Reward: 18.161858974358974. Std of Reward: 7.314722404295779.
Step: 5030000. Mean Reward: 18.45686274509804. Std of Reward: 6.805483800985408.
Step: 5040000. Mean Reward: 18.366993464052285. Std of Reward: 7.1851407202567

Saved Model
Step: 5910000. Mean Reward: 18.123856209150325. Std of Reward: 6.856948875028092.
Step: 5920000. Mean Reward: 17.788636363636364. Std of Reward: 6.515674215164399.
Step: 5930000. Mean Reward: 17.108387096774194. Std of Reward: 7.431940265382964.
Step: 5940000. Mean Reward: 17.68474025974026. Std of Reward: 6.980686082276458.
Step: 5950000. Mean Reward: 17.825806451612902. Std of Reward: 6.982917762559433.
Saved Model
Step: 5960000. Mean Reward: 17.333774834437087. Std of Reward: 7.886052214223501.
Step: 5970000. Mean Reward: 18.49967741935484. Std of Reward: 6.8143172255759.
Step: 5980000. Mean Reward: 17.835064935064935. Std of Reward: 7.310948758473841.
Step: 5990000. Mean Reward: 17.9375. Std of Reward: 7.062973901193983.
Step: 6000000. Mean Reward: 17.80588235294118. Std of Reward: 7.327609042983445.
Saved Model


KeyboardInterrupt: 

### Export the trained Tensorflow graph
Once the model has been trained and saved, we can export it as a .bytes file which Unity can embed.

In [6]:
export_graph(model_path, env_name)

INFO:tensorflow:Restoring parameters from ./models/ppo\model-6000000.cptk


INFO:tensorflow:Restoring parameters from ./models/ppo\model-6000000.cptk


INFO:tensorflow:Froze 4 variables.


INFO:tensorflow:Froze 4 variables.


Converted 4 variables to const ops.


In [7]:
env.close()